In [ ]:
# Importar librerías

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

In [ ]:


# Cargar datos
df = pd.read_csv("datos_ipsa.csv")  # Debe tener "Date" y "Price"
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date")  # Asegurar orden temporal

# Normalizar precios entre 0 y 1
scaler = MinMaxScaler(feature_range=(0, 1))
df["Price"] = scaler.fit_transform(df["Price"].values.reshape(-1, 1))

# Crear datos de entrenamiento con ventanas de tiempo
def crear_secuencias(datos, ventana=30):
    X, y = [], []
    for i in range(len(datos) - ventana):
        X.append(datos[i:i+ventana])
        y.append(datos[i+ventana])
    return np.array(X), np.array(y)

ventana = 30  # Usar los últimos 30 días para predecir
X, y = crear_secuencias(df["Price"].values, ventana)

# Dividir datos en entrenamiento y prueba
X_train, X_test = X[:int(0.8*len(X))], X[int(0.8*len(X)):]
y_train, y_test = y[:int(0.8*len(y))], y[int(0.8*len(y)):]

# Reshape para LSTM (samples, timesteps, features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Definir modelo LSTM
modelo = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])

# Compilar modelo
modelo.compile(optimizer="adam", loss="mean_squared_error")

# Entrenar modelo
modelo.fit(X_train, y_train, batch_size=16, epochs=20, validation_data=(X_test, y_test))

# Hacer predicción para mañana
ultimos_datos = df["Price"].values[-ventana:].reshape(1, ventana, 1)
prediccion = modelo.predict(ultimos_datos)
prediccion_real = scaler.inverse_transform(prediccion.reshape(-1, 1))

print(f"Predicción del IPSA para mañana: {prediccion_real[0][0]:.2f}")

# Graficar predicciones vs. reales
predicciones = modelo.predict(X_test)
predicciones_real = scaler.inverse_transform(predicciones.reshape(-1, 1))
y_test_real = scaler.inverse_transform(y_test.reshape(-1, 1))

plt.figure(figsize=(10, 5))
plt.plot(y_test_real, label="Real")
plt.plot(predicciones_real, label="Predicho", linestyle="dashed")
plt.legend()
plt.title("Predicción del IPSA con LSTM")
plt.show()
